# Spam Filtering con Naive Bayes
Per questa esercitazione dovrai utilizzare l'intero dataset di sms di spam per creare un classificare di spam utilizzando un algoritmo Naive Bayes.
#### Task:
- Scarica il [dataset da Kaggle](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset) (richiede un account gratuito), puoi farlo anche utilizzando le API.
- Processa il dataset per ottenere un dataframe con la stessa struttura di quello visto nelle lezioni di pratica.
- Costruisci e valuta i tuoi modelli ottimizzando le metriche che reputi corretto ottimizzare in base al problema affrontato.
- Una volta selezionato il modello finale, seleziona 3 email spam e 3 email non spam dalla tua casella di posta e prova ad usare il modello per classificarle. (n.b va bene anche se il tuo modello non le classifica tutte correttamente, ricorda che il dataset è di sms non di email)

## Import del dataset

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("spam.csv",usecols=["v1","v2"],encoding="latin_1")

print(df.shape)
df.head()

(5572, 2)


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Verifica dei valori mancanti

In [2]:
df.isna().sum()

v1    0
v2    0
dtype: int64

Bene, non ci sono valori mancanti

Avendo a che fare con il testo, devo creare la bag of words. Posso usare sklearn per farlo. Ma prima devo dividere in train e test.

In [3]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 176
X = df["v2"].values
y = df["v1"].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=RANDOM_SEED)


Creo un dizionario di 2000 parole al massimo (valore troppo alto o troppo basso? Bho, vedi soluzione dopo)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(stop_words="english",
                                    encoding="latin_1",
                                    max_features=2000
                                  )
X_train = count_vectorizer.fit_transform(X_train)
X_test = count_vectorizer.transform(X_test)

In [5]:
#Per verifica
print(X_train.shape)
print(X_test.shape)

(3900, 2000)
(1672, 2000)


## Creazione modello
Avendo a che fare con il testo, si può provare classificatore bayesiano, di tipo Bernoulli o Multinomial/Complement.

In [8]:
#serve dopo
from sklearn.metrics import classification_report

In [9]:
#Bernoulli
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(X_train,y_train)

#In questo caso trattiamo ogni feature della bag of words come 
#"parola presente? Sì/No"

y_pred_bernoulli=bnb.predict(X_test)
print(classification_report(y_test,y_pred_bernoulli))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1429
        spam       1.00      0.86      0.92       243

    accuracy                           0.98      1672
   macro avg       0.99      0.93      0.96      1672
weighted avg       0.98      0.98      0.98      1672



In [23]:
#multinomial
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB() #default laplace smoothing
mnb.fit(X_train,y_train)

#In questo caso trattiamo ogni feature della bag of words come 
#conteggio delle occorrenze di ogni parola

y_pred_multinomial=mnb.predict(X_test)
print(classification_report(y_test,y_pred_multinomial))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1429
        spam       0.95      0.94      0.94       243

    accuracy                           0.98      1672
   macro avg       0.97      0.96      0.97      1672
weighted avg       0.98      0.98      0.98      1672



L'accuracy è la stessa, però migliora l'f1-score (perchè migliora la recall, anche se la precision un po' preggiora). 
Qui contare le occorrenze porta ad un miglioramento rispetto al caso di solo "presente/assente"

Per vedere se il modello Complement naive bayes può essere utile, vediamo se il dataset è sbilanciato

In [22]:
class_details=df["v1"].value_counts()
print(class_details)

print(f"Classe positiva sono il {class_details[1]/class_details.sum()*100:.2f}% dei valori totali")

v1
ham     4825
spam     747
Name: count, dtype: int64
Classe positiva sono il 13.41% dei valori totali


In effetti il dataset è sbilanciato verso la classe negativa ("ham").

In [24]:
#Complement
from sklearn.naive_bayes import ComplementNB

cnb = ComplementNB()
cnb.fit(X_train,y_train)

#stesso significato del caso multinomial, 
#ma tengo conto dello sbilanciamento

y_pred_complement = cnb.predict(X_test)
print(classification_report(y_test,y_pred_complement))


              precision    recall  f1-score   support

         ham       0.99      0.96      0.98      1429
        spam       0.80      0.96      0.87       243

    accuracy                           0.96      1672
   macro avg       0.90      0.96      0.92      1672
weighted avg       0.96      0.96      0.96      1672



Secondo me il modello migliore è l'ultimo perchè ha recall (sensibilità) più alta, così becco più facilmente gli spam (o dovrei privilegiare la specificità? Bho).

EDIT: meglio tenere conto di entrambe le classi, quindi qui privilegio l'accuratezza (vince il multinomial).

## Prova con e-mail

Vedi soluzione